In [2]:
import pandas as pd
import spacy

In [3]:
import os
import sklearn
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import enchant
from sklearn.neighbors import NearestNeighbors
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
data = pd.read_csv('go_V.csv', sep = ';')

In [11]:
PIPELINE = ['tagger', 'parser', 'ner']

In [14]:
nlp = spacy.load('xx_ent_wiki_sm')

for name in PIPELINE:
    component = nlp.create_pipe(name)

In [15]:
nlp.pipeline

[('ner', <spacy.pipeline.EntityRecognizer at 0x7f545daf5728>)]

In [16]:
%%time

docs = [nlp(txt) for txt in data.text[:50]]

CPU times: user 2.6 s, sys: 1.88 s, total: 4.48 s
Wall time: 1.18 s


In [17]:
df_pos = pd.DataFrame(columns=['Text', 'Lemma', 'POS', 'TAG', 'DEP', 'Shape', 'Stop Word'])

for token in docs[10]:
    df_pos = df_pos.append({'Text': token.text, 'Lemma': token.lemma_, 'POS': token.pos_, 'TAG': token.tag_, 
                            'DEP': token.dep_, 'Shape': token.shape_, 'Stop Word': token.is_stop}, ignore_index=True)
    
df_pos.head(1000000)

,Text,Lemma,POS,TAG,DEP,Shape,Stop Word
0,Бывший,Бывший,,,,Xxxxx,False
1,футболист,футболист,,,,xxxx,False
2,сборной,сборной,,,,xxxx,False
3,России,России,,,,Xxxxx,False
4,Александр,Александр,,,,Xxxxx,False
5,Мостовой,Мостовой,,,,Xxxxx,False
6,в,в,,,,x,False
7,беседе,беседе,,,,xxxx,False
8,с,с,,,,x,False
9,корреспондентом,корреспондентом,,,,xxxx,False


In [19]:
from spacy import displacy

In [20]:
for doc in docs:
    displacy.render(doc, style='ent', jupyter=True)
    print('=============================================================================================================')

In [21]:
df_ent = pd.DataFrame(columns=['Text', 'Start Char', 'End Char', 'Label'])

for ent in docs[10].ents:
    df_ent = df_ent.append({'Text': ent.text, 'Start Char': ent.start_char, 'End Char': ent.end_char, 'Label': ent.label_},
                           ignore_index=True)
    
df_ent.head(10000000)

,Text,Start Char,End Char,Label
0,России Александр Мостовой,25,50,ORG
1,Ленты.ру,79,87,PER
2,Посмотрев,314,323,MISC
3,Кубок России,356,368,LOC
4,Футболисты,417,427,LOC
5,Это,472,475,PER
6,Непонятно чего,651,665,MISC
7,Что,675,678,PER
8,Мостовой. Чемпионат России,707,733,MISC
9,Матч,786,790,PER


In [9]:
print([(ent.text, ent.label) for ent in doc.ents])

[('водные', 381), ('Флориды', 381), ('—', 4317129024397789502), ('PLoS One', 7654241940133152407), ('Флориды', 383), ('По', 383), ('Зоологи', 7654241940133152407), ('выгоду.', 381), ('Ученые', 383), ('Оказалось', 4317129024397789502), ('Дело', 4317129024397789502), ('Кроме', 383), ('Зоологи', 7654241940133152407)]


In [10]:
for token in doc:
    print("{0}\t{1}\t{2}\t{3}\t{4}\t{5}\t{6}\t{7}".format(
        token.text,
        token.idx,
        token.lemma_,
        token.is_punct,
        token.is_space,
        token.shape_,
        token.poЯ МИША ДУРАЧОК АЗАЗАs_,
        token.tag_
    ))

Ученые	0	Ученые	False	False	Xxxxx		
выяснили	7	выяснили	False	False	xxxx		
что	16	что	False	False	xxx		
водные	20	водные	False	False	xxxx		
и	27	и	False	False	x		
околоводные	29	околоводные	False	False	xxxx		
птицы	41	птицы	False	False	xxxx		
в	47	в	False	False	x		
болотах	49	болотах	False	False	xxxx		
Флориды	57	Флориды	False	False	Xxxxx		
специально	65	специально	False	False	xxxx		
строят	76	строят	False	False	xxxx		
гнезда	83	гнезда	False	False	xxxx		
поблизости	90	поблизости	False	False	xxxx		
от	101	от	False	False	xx		
аллигаторов.	104	аллигаторов.	False	False	xxxx.		
Рептилии	117	Рептилии	False	False	Xxxxx		
выполняют	126	выполняют	False	False	xxxx		
функции	136	функции	False	False	xxxx		
телохранителей	144	телохранителей	False	False	xxxx		
отпугивая	159	отпугивая	False	False	xxxx		
от	169	от	False	False	xx		
птиц	172	птиц	False	False	xxxx		
других	177	других	False	False	xxxx		
хищников	184	хищников	False	False	xxxx		
—	193	--	True	False	—	PUNCT	
зато	195	зато	False	False	xxxx		


In [22]:
!python3 -m pip install pymorphy2==0.8

  Using cached https://files.pythonhosted.org/packages/a3/33/fff9675c68b5f6c63ec8c6e6ff57827dda28a1fa5b2c2d727dffff92dd47/pymorphy2-0.8-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/6a/84/ff1ce2071d4c650ec85745766c0047ccc3b5036f1d03559fd46bb38b5eeb/DAWG_Python-0.7.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/02/51/2465fd4f72328ab50877b54777764d928da8cb15b74e2680fc1bd8cb3173/pymorphy2_dicts-2.4.393442.3710985-py2.py3-none-any.whl
